In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import dill
import pandas as pd
import polars as pl
import numpy as np
from scipy.sparse import csr_matrix

import lightfm

In [3]:
from otto_utils import *

In [6]:
df_test = pl.read_parquet(TEST_PROCESSED, use_pyarrow=True)

In [40]:
df_test["session"].value_counts(sort=True)

session,counts
i64,u32
14086824,458
13352358,433
13242700,421
13234219,407
13346726,406
13932091,405
12937717,403
14337964,392
13944570,389


In [4]:
# load
val_df_train = pl.read_parquet(VALIDATION_PATH / "train.parquet", use_pyarrow=True)
val_df_valid_input = pl.read_parquet(VALIDATION_PATH / "valid.parquet", use_pyarrow=True)
val_df_valid_targets = pl.read_parquet(VALIDATION_PATH / "test_labels.parquet", use_pyarrow=True)

In [9]:
val_df_valid_input.groupby("type").count()

type,count
i64,u32
0,5979477
2,68895
1,590001


In [30]:
df_last_20_actions = (
    val_df_valid_input
    .groupby(["session", "aid", "type"])
    .agg([
        pl.min("ts")
    ])
    .sort("ts", reverse=True)
    .groupby("session")
    .agg([
#         pl.list("ts").alias("ts"),
        pl.list("aid").alias("labels"),
    ])
    .with_columns([
        pl.col("labels").arr.head(20).alias("labels"),
    ])
)

In [31]:
df_last_20_actions

session,labels
i64,list[i64]
12775296,[280584]
12651008,"[1611032, 1385064, ... 691511]"
12537216,"[381147, 361104, ... 874858]"
11784832,"[1111967, 232749, 1468783]"
12334464,"[886705, 1660379]"
11724416,[847845]
12393280,[194612]
12200128,"[1105832, 760336, ... 856111]"
11654528,"[270035, 513192]"


In [32]:
df_submission = (
    pl.concat([
        (
            df_last_20_actions
#             .with_column(.alias("type"))
            .with_column(pl.concat_str([pl.col("session"), pl.lit(f"_{t}")]).alias("session_type"))
            .with_column(pl.col("labels").apply(lambda x: " ".join(str(aid) for aid in x)))
            .select(["session_type", "labels"])
        )
        for t in ["clicks", "carts", "orders"]
    ])
)

In [33]:
df_submission

session_type,labels
str,str
"""12775296_click...","""280584"""
"""12651008_click...","""1611032 138506..."
"""12537216_click...","""381147 361104 ..."
"""11784832_click...","""1111967 232749..."
"""12334464_click...","""886705 1660379..."
"""11724416_click...","""847845"""
"""12393280_click...","""194612"""
"""12200128_click...","""1105832 760336..."
"""11654528_click...","""270035 513192"""


In [34]:
calc_valid_score(df_submission)

/app/notebooks/otto/otto_utils.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_labels["ground_truth"].loc[labels_null_idx] = (


validation score: 0.4505167349101711
recall per type: type
carts     0.291750
clicks    0.325259
orders    0.550776
dtype: float64
